**Important: Before you start, please add your name and id to the notebook file name and add it here as well**

Example: "notebook2_John_Doe_123123123.ipynb"

**Names:**

1. Ofir Gormes 208575068

2. Eden Bresler 209424589

# Part 1

# Scoring matrix BLOSUM62

Install `biopython` using the following steps:
1. Run "Anaconda Prompt" as administrator (from the Start Menu)
2. Type `conda install -c anaconda biopython`

if you're using Google Colab, install `biopython` using the following command:
`!pip install biopython` (uncomment it in the command below)

Obtain BLOSUM62 scoring matrix for amino acids as follows:

In [176]:
# uncomment the following line if you're using Google Colab:
!pip install biopython

from Bio.Align import substitution_matrices
matrix = substitution_matrices.load("BLOSUM62")

For example, the score for aligning amino acid `S` versus `P` is:

In [177]:
matrix['S', 'P']

-1.0

In [178]:
matrix['I', 'I']

4.0

In [179]:
matrix['H', 'K']

-1.0

# BLAST
The input for BLAST is a **Database (DB)** sequence, and one or more **query** sequences.  
In addition, a scoring matrix and parameters $k$, $T$, and $X$.

## 1. **Database pre-process**:
  
  Mapping each $k$-mer (a substrings of length $k$) from the **DB** to all its indexes in the **DB** sequence.
  This is done once for the DB sequence, and used for multiple queries.
  
  Complete the function `build_db` that creates a dictionary as follows:   
  **key** - a $k$-mer from `db`, **value** - a list of start indexes of the $k$-mer in `db`

In [180]:
def build_db(db, k):

    db_dict = {}
    for i in range(0, len(db) - k + 1):
      if db[i: i+k] not in db_dict.keys():
        db_dict[db[i: i+k]] = [i]
      else:
        db_dict[db[i: i+k]].append(i)


    return db_dict

In [181]:
db_dict = build_db('AACGTAAAC', 3)

In [182]:
assert db_dict == {'AAC': [0, 6], 'ACG': [1], 'CGT': [2], 'GTA': [3], 'TAA': [4], 'AAA': [5]}

## 2. **Query processing**:
  
###  i. For each $k$-mer in the query, make a list of neighbor words.
  > **Reminder**: A neighbor word $𝑠′$ of a $k$-mer $𝑠$ is a word of length $|s|=k$, with $score(s,s')\geq T$ (using a scoring matrix). $T$ is a chosen threshold.


#### First, a helper function.   
Complete the function `align` that gets two sequences of the same length and a scoring matrix, and returns the score of alignment without indels.

In [183]:
def align(seq1, seq2, scoring_matrix):
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be with the same length")
    score = 0
    for i in range(len(seq1)):
      score += scoring_matrix[seq1[i], seq2[i]]
    return score

In [184]:
assert align('YLKSHFCTA', 'YLNAKKFVH', matrix) == 4

#### Finding neighbors
Let's look at the alphabet of amino acids

In [185]:
alphabet = matrix.alphabet

In [186]:
alphabet

'ARNDCQEGHILKMFPSTWYVBZX*'

We will use the first 20 amino-acid symbols, the last 4 signify a combination of two or more amino acids

In [187]:
ALPHABET = matrix.alphabet[:20]
ALPHABET

'ARNDCQEGHILKMFPSTWYV'

Complete the function `find_neighbors_rec` that finds all neighbors of a $k$-mer

> Description of `find_neighbors_rec`:    
In each step of the recursion, assume that `neighbor[pos:] == kmer[pos:]`   
**Stop condition**: If `pos` is equal to the length of the `kmer`, we add the `neighbor` to `neighbors` list   
**Otherwise**, we will try to replace the char in index `pos` with each letter from the `alphabet` (including itself)    
The new score will be obtained by removing the alignment score of `kmer[pos]` with itself, and adding the alignment score of `kmer[pos]` with the new letter  
If the new score is $>=T$, we will continue with the recursion (and go to the next position)

In [188]:
def find_neighbors(kmer, scoring_matrix, alphabet, T):

    neighbors = []
    max_score = align(kmer, kmer, scoring_matrix)

    if max_score >= T:
        find_neighbors_rec(kmer, kmer, 0, max_score, alphabet, neighbors, scoring_matrix, T)

    return neighbors

def find_neighbors_rec(kmer, neighbor, pos, curr_score, alphabet, neighbors, scoring_matrix, T):
    if pos == len(kmer):
      neighbors.append(neighbor)
    else:
      for letter in alphabet:
        neighbor = neighbor[:pos] + letter + neighbor[pos+1:]
        new_score = curr_score - align(kmer[pos], kmer[pos], scoring_matrix) + align(kmer[pos], letter, scoring_matrix)
        if new_score >= T:
          find_neighbors_rec(kmer, neighbor, pos + 1, new_score, alphabet, neighbors, scoring_matrix, T)

These are all possible neighbors of `YI`

In [189]:
neighbors = find_neighbors('YI', matrix, ALPHABET, -50)

This number should be equal to $20^2=400$

In [190]:
assert len(neighbors) == 400

In [191]:
neighbors[:10]

['AA', 'AR', 'AN', 'AD', 'AC', 'AQ', 'AE', 'AG', 'AH', 'AI']

In [192]:
neighbors = find_neighbors('YI', matrix, ALPHABET, 6)

In [193]:
neighbors

['HI', 'FI', 'FV', 'WI', 'YA', 'YC', 'YI', 'YL', 'YM', 'YF', 'YT', 'YY', 'YV']

In [194]:
assert set(neighbors) == set(['HI', 'FI', 'FV', 'WI', 'YA', 'YC', 'YI', 'YL', 'YM', 'YF', 'YT', 'YY', 'YV'])

# Part 2

  
### ii. Finding HSPs
For each neighbor, find the indexes of its appearances in the DB (**hits**), using the dictionary constructed in stage **1**.

A **hit** is called an HSP (High-Scoring Segment Pair). You are provided with the class `HSP`.

In [195]:
class HSP():
    """Represents an alignment with score >=T between a kmer in the query and a kmer in the db"""

    def __init__(self, query_start, query_end, db_start, db_end, score):
        self.query_start = query_start
        self.query_end = query_end
        self.db_start = db_start
        self.db_end = db_end
        self.score = score

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            is_query_eq = self.query_start == other.query_start and self.query_end == other.query_end
            is_db_eq = self.db_start == other.db_start and self.db_end == other.db_end
            return is_query_eq and is_db_eq and self.score == other.score
        return False

    def __ne__(self, other):
        """Define a non-equality test"""
        return not self.__eq__(other)

    def __hash__(self):
        return hash(str((self.query_start, self.query_end, self.db_start, self.db_end, self.score)))

    def __str__(self):
        return f'''
                Score: {self.score}
                Query: [{self.query_start}, {self.query_end}]
                DB: [{self.db_start}, {self.db_end}]
                '''
    def __repr__(self):
        return self.__str__()

Complete the function `get_hsps` that returns all hsps between a $k$-mer in `query` and a $k$-mer in db

In [196]:
def get_hsps(query, db_dict, k, scoring_matrix, T):

    hsps = []
    for i in range(len(query)):
      kmer = query[i: i + k]
      neighbors_k = find_neighbors(kmer, scoring_matrix, alphabet, T)
      for neighbor in neighbors_k:
        if neighbor in db_dict.keys():
          neighbor_indexes = db_dict[neighbor]
          for index in neighbor_indexes:
            score = align(neighbor, kmer, scoring_matrix)
            hsp = HSP(i, i + k - 1, index, index + k-1, score)
            hsps.append(hsp)
    return hsps

In [197]:
k = 5
T = 23
db = 'VTSAQELRGCTVINGSLIINIRGGNNLAAELEANLG'
query = 'AANIEGGNNAA'

db_dict = build_db(db, k)
hsps = get_hsps(query, db_dict, k, matrix, T)

In [198]:
expected_hsps = set([HSP(query_start=5,query_end=9,db_start=22,db_end=26,score=23.0),
                     HSP(query_start=4,query_end=8,db_start=21,db_end=25,score=24.0)])
print(expected_hsps)

{
                Score: 23.0
                Query: [5, 9]
                DB: [22, 26]
                , 
                Score: 24.0
                Query: [4, 8]
                DB: [21, 25]
                }


In [199]:
assert set(hsps) == expected_hsps

## 3. **Extension of HSPs**:
    
Extend each HSP found in **stage 2** to obtain MSPs (Maximal Scoring Pairs). Extension stops when the extended HSP score drops X below the maximal score obtained so far.

In the next function, you can use the imported method `copy`    
`copy` performs a deep copy of an object

In [200]:
from copy import copy

In [201]:
hsp1 = HSP(5,10,22,27,23.0)
hsp2 = copy(hsp1)

In [202]:
hsp1


                Score: 23.0
                Query: [5, 10]
                DB: [22, 27]
                

In [203]:
hsp2.score = 50

`hsp1` didn't change after changing `hsp2`

In [204]:
print("hsp1", hsp1)
print("hsp2", hsp2)

hsp1 
                Score: 23.0
                Query: [5, 10]
                DB: [22, 27]
                
hsp2 
                Score: 50
                Query: [5, 10]
                DB: [22, 27]
                


You are given the function `extend_hsp`, the hsp is first extended to the left, and the best scoring extension is returned.   
Then, the left best scoring extension (it can also be the initial hsp) is given to the function `extend_right`

Complete the functions `extend_left` and `extend_right`

In [205]:
def extend_left(query, db, hsp, scoring_matrix, X):
    """returns the left extension with the maximal score"""

    msp = copy(hsp)
    max_score = hsp.score
    score = max_score
    q_start = hsp.query_start
    db_start = hsp.db_start
    q_end = hsp.query_end
    db_end = hsp.db_end
    while score >= max_score - X:
      if score > max_score:
        max_score = score
        msp.query_start = q_start
        msp.db_start = db_start
        msp.score = score
      if q_start <= 0 or db_start <= 0:
        break
      q_start -= 1
      db_start -= 1
      score += scoring_matrix[query[q_start], db[db_start]]

    return msp

In [206]:
def extend_right(query, db, hsp, scoring_matrix, X):
    """returns the right extension with the maximal score"""

    msp = copy(hsp)
    max_score = hsp.score
    score = max_score
    q_start = hsp.query_start
    db_start = hsp.db_start
    q_end = hsp.query_end
    db_end = hsp.db_end
    while score >= max_score - X:
      if score > max_score:
        max_score = score
        msp.query_end = q_end
        msp.db_end = db_end
        msp.score = score
      if q_end >= len(query) - 1 or db_end >= len(db) - 1:
        break
      q_end += 1
      db_end += 1
      score += scoring_matrix[query[q_end], db[db_end]]

    return msp

In [207]:
def extend_hsp(query, db, hsp, scoring_matrix, X):

    msp_left = extend_left(query, db, hsp, scoring_matrix, X)
    msp = extend_right(query, db, msp_left, scoring_matrix, X)

    return msp

In [208]:
k = 5
T = 24
db = 'VTSAQELRGCTVINGSLIINIRGGNNLAAELEANLG'
db_dict = build_db(db, k)
query = 'AANIEGGNNAAA'

hsps = get_hsps(query, db_dict, k, matrix, T)

In [209]:
hsp = hsps[0]
hsp


                Score: 24.0
                Query: [4, 8]
                DB: [21, 25]
                

In [210]:
assert hsp == HSP(4, 8, 21, 25, 24.0)

In [211]:
X = 20

msp = extend_hsp(query, db, hsp, matrix, X)
msp


                Score: 41.0
                Query: [2, 11]
                DB: [19, 28]
                

In [212]:
assert msp == HSP(2, 11, 19, 28, 41.0)

## 4. **Find all MSPs**:

Combine stages 1-3.  

In [213]:
def find_msps(query, db, k, scoring_matrix, T, X):

    msps = set()
    db_dict = build_db(db, k)
    hsps = get_hsps(query, db_dict, k, scoring_matrix, T)
    for hsp in hsps:
      msp = extend_hsp(query, db, hsp, scoring_matrix, X)
      msps.add(msp)

    return msps

In [214]:
k = 4
T = 20
X = 20
db = 'AINIEGGNCTVINGSLIINIRNNAAAAELEANLG'
query = 'AANIEGGNNAAA'

msps = find_msps(query, db, k, matrix, T, X)

In [215]:
msps

{
                 Score: 24.0
                 Query: [7, 11]
                 DB: [21, 25]
                 ,
 
                 Score: 36.0
                 Query: [0, 7]
                 DB: [0, 7]
                 }

Get top scoring MSPs of the query.

In [216]:
def get_top_msps(msps, top):
    sorted_msps = sorted(list(msps), reverse=True, key=lambda msp:msp.score)
    return sorted_msps[:top]

In [217]:
msps = get_top_msps(msps, 1)
msp = msps[0]
msp


                Score: 36.0
                Query: [0, 7]
                DB: [0, 7]
                

In [218]:
assert msp == HSP(0, 7, 0, 7, 36.0)

# Part 3
### Input files:
- query1.fasta
- db1.fasta

*query1.fasta* contains a protein query sequence, and *db1.fasta* contains the database that represents the concatanations of different proteins.

In this part you will run your BLAST using the query in *query1.fasta* and the Database in *db1.fasta*.


We will use the function `read_seq_file` to read the files *query1.fasta* and *db1.fasta*

**Important:** If you're using Google Colab, you have to upload the files to your session storage. Click on "Files" on the left side (the folder icon). After it click on "Upload to session storage" and choose the files you would like to upload.

Please note that you have to upload the files every time you open a session.

In [219]:
def read_seq_file(seq_file):
    seq_id = ''
    seq = ''
    with open(seq_file) as f:
        for line in f:
            if line.startswith('>'):
                seq_id = line.strip()[1:]
            else:
                seq += line.strip()

        return seq_id, seq

In [220]:
query_id, query = read_seq_file('query1.fasta')

In [221]:
len(query)

1382

In [222]:
query[:10]

'MATGGRRGAA'

In [223]:
db_id, db = read_seq_file('db1.fasta')

In [224]:
len(db)

9520

BLAST is a hueristic. To find out the real alignment between the `query` and the `db`, we will use local alignment algorithm.

Example:

In [225]:
from Bio import pairwise2

db_small = 'KKKKEAVLA'
query_small = 'EVL'

alignments = pairwise2.align.localds(db_small, query_small, matrix, -4, -4)

align1, align2, score, start, end = alignments[0]

print(align1)
print(align2)
print(f'Location: [{start}, {end}]')

KKKKEAVLA
----E-VL-
Location: [4, 8]


Now using the `query` and `db` from the files

In [226]:
from datetime import datetime

start_time = datetime.now()

alignments = pairwise2.align.localds(db, query, matrix, -4, -4)

timedelta = datetime.now() - start_time

It should take about 20 seconds

In [227]:
print(timedelta)

0:00:22.843850


In [228]:
align1, align2, score, start, end = alignments[0]

In [229]:
print(f'Score: {score}, [{start}, {end}]')

Score: 6987.0, [2999, 4366]


Now let's see if our BLAST is good enough

In [230]:
k = 4
T = 23
X = 8

start_time = datetime.now()

msps = find_msps(query, db, k, matrix, T, X)

timedelta = datetime.now() - start_time

It should be much faster

In [231]:
print(timedelta)

0:00:01.949484


In [232]:
get_top_msps(msps, 3)

[
                 Score: 3835.0
                 Query: [33, 753]
                 DB: [3016, 3736]
                 ,
 
                 Score: 3088.0
                 Query: [783, 1381]
                 DB: [3753, 4351]
                 ,
 
                 Score: 127.0
                 Query: [754, 792]
                 DB: [3725, 3763]
                 ]

Discuss the results.
  - Is there an MSP that is identical to the optimal Local Alignment?
  - If not, why do you think so?
  - How can BLAST be improved, in order to get more similar results to Local Alignment?

**Answer:**

*   No
*   Blast is a heuristic algorithm, which means it won't always find the optimal alignment, but will find  good alignments (faster). In contrast to local alignment that promise to find the optimal alignment.
* Increasing T (but not too much) will get better solutions because the treshold will be greater and we'll get the alignments that are more optimal.






# Part 4

### Input files:
- query2.fasta
- db2.fasta

*query2.fasta* contains a query protein alpha globin, and db2 is the database (DB), containing 3 homologs of the query.

We will run local alignment algorithm using the query and the database.
   - After running the algorithm, we will get **start** and **end** indices of the local alignment in the database.
   - We will remove the substring DB[start:end] from the DB string and run local alignment again, using the new DB string.
   - This will help you find locations of all 3 homologs of the query in the database.  

In [233]:
query2_id, query2 = read_seq_file('query2.fasta')

In [234]:
db2_id, db2 = read_seq_file('db2.fasta')

In [235]:
len(db2)

10437

In [236]:
alignments = pairwise2.align.localds(db2, query2, matrix, -4, -4)

align1, align2, *alignment1 = alignments[0]

Remove a substring that matched `query2` from `db2`

In [237]:
score, start, end = alignment1
alignment1

[540.0, 2395, 2537]

Remove a substring that matched `query2` from `db2`

In [238]:
db2 = db2[:start] + db2[end:]

In [239]:
len(db2)

10295

Align `query2` against the new `db2`

In [240]:
alignments = pairwise2.align.localds(db2, query2, matrix, -4, -4)

align1, align2, *alignment2 = alignments[0]

In [241]:
score, start, end = alignment2
alignment2

[213.0, 1348, 1494]

Remove another substring that matched `query2` from `db2`

In [242]:
db2 = db2[:start] + db2[end:]

Align `query2` against the new `db2`

In [243]:
alignments = pairwise2.align.localds(db2, query2, matrix, -4, -4)

align1, align2, *alignment3 = alignments[0]

In [244]:
score, start, end = alignment3
alignment3

[204.0, 230, 378]

All homolog locations

In [245]:
print(f'{alignment1}\n{alignment2}\n{alignment3}')

[540.0, 2395, 2537]
[213.0, 1348, 1494]
[204.0, 230, 378]


In [246]:
k = 4
T = 20
X = 100

db2_id, db2 = read_seq_file('db2.fasta')
msps = find_msps(query2, db2, k, matrix, T, X)
get_top_msps(msps, 5)

[
                 Score: 267.0
                 Query: [32, 87]
                 DB: [2440, 2495]
                 ,
 
                 Score: 183.0
                 Query: [87, 127]
                 DB: [2496, 2536]
                 ,
 
                 Score: 154.0
                 Query: [0, 33]
                 DB: [2395, 2428]
                 ,
 
                 Score: 138.0
                 Query: [38, 97]
                 DB: [1404, 1463]
                 ,
 
                 Score: 111.0
                 Query: [41, 82]
                 DB: [289, 330]
                 ]

Look at **5 top** scoring MSPs obtained with the parameters above.
   - Did you find all 3 homologs in the database?
   - Measure Precision and Sensitivity (If we found an MSP that is contained in a local alignment, it is considered to be a found homolog).

**Answer:**

*   Yes
*   Sensitivity: 1, Precision: 1



Now we will change parameter $T$

In [247]:
k = 4
T = 26
X = 100

db2_id, db2 = read_seq_file('db2.fasta')
msps = find_msps(query2, db2, k, matrix, T, X)
get_top_msps(msps, 4)

[
                 Score: 39.0
                 Query: [2, 17]
                 DB: [1351, 1366]
                 ,
 
                 Score: 38.0
                 Query: [2, 17]
                 DB: [233, 248]
                 ]

You should get only two MSPs with the parameters above.   
- Did you find all 3 homologs in the database?
- Measure Precision and Sensitivity (If we found an MSP that is contained in a local alignment, it is considered to be a found homolog).

**Answer:**

*   No
*   Sensitivity: 2/3 Precision: 1


- Discuss how parameters $T$ and $X$ affect Precision and Sensitivity.

**Answer:**

* T:
  * precision - the more T is greater, then precision is greater. And vise verca
  * sensitivity -the more T is greater, then sensitivity gets lower. And vise verca
* X:
  * precision - the more X is greater then, precision gets lower. And vise verca
  * sensitivity - the more X is greater then, sensitivity is greater. And vise verca